In [ ]:
def generate_prompt(instruction: str, input_ctxt: str = None) -> str:
    if input_ctxt:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_ctxt}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""


In [1]:
!pip install transformers
!pip install sentencepiece

import torch
from transformers import GenerationConfig, LlamaTokenizer, LlamaForCausalLM

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 9.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.3 MB/s eta 0:00:00


In [2]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.4.0
alabaster                     0.7.13
albumentations                1.2.1
altair                        4.2.2
anyio                         3.6.2
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.15.1
astropy                       5.2.2
astunparse                    1.6.3
attrs                         23.1.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.12.1
backcall                      0.2.0
beautifulsoup4                4.11.2
bleach                        6.0.0
blis                          0.7.9
blosc2                        2.0.0
bokeh                         2.4.3
branca                        0.6.0
CacheControl                  0.12.11
cached-property               1.5.2
cachetools                    5.3.0
cata

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("chainyo/alpaca-lora-7b")
model = LlamaForCausalLM.from_pretrained(
    "chainyo/alpaca-lora-7b",
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
generation_config = GenerationConfig(
    temperature=0.2,
    top_p=0.75,
    top_k=40,
    num_beams=4,
    max_new_tokens=128,
)

model.eval()
if torch.__version__ >= "2":
    model = torch.compile(model)

questions = [
    'What does μονογενής mean?',
    'What is the grammatical case of μονογενής?',
    'What is the semantic domain information of μονογενής?',
    'What is the lemma form of μονογενής?',
    "What is the Strong's number of μονογενής?",
    'What are the 3 discourse features that this word functions within?',
    'What is the Louw-Nida semantic domain entry for this word?',
    'How does the verb focus contribute to the meaning of the clause?'
]

input_ctxt = '''
and its grammatical case is accusative. μονογενής is a adj, and its semantic 
domain information is 058007. The grammatical gender of this word is masculine, 
meaning “only begotten.” The lemma form of this word is μονογενής, 
The Louw-Nida semantic domain entry for this word is 58.52. and it is 
classified as a A-ASM and its grammatical number is singular. with a Strong’s 
number of 3439. This word functions within 3 discourse feature(s), including  
Focus+, which is defined as Constituents placed in P2 to give them focal 
prominence. Verb Focus+, which is defined as Verb in final position in clause 
demonstrates verb focus. Main clauses, which is defined as Main clauses are the 
top-level clauses in a sentence. 
They are the clauses that are not embedded in other clauses.
'''

for question in questions:
  prompt = generate_prompt(question, input_ctxt)
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids
  input_ids = input_ids.to(model.device)

  with torch.no_grad():
      outputs = model.generate(
          input_ids=input_ids,
          generation_config=generation_config,
          return_dict_in_generate=True,
          output_scores=True,
      )

  response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
  print(response)
  print('\n')
  print('--------------------------------------------------------------------')
  print('\n')


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28

Loading checkpoint shards:   0%|          | 0/39 [00:00<?, ?it/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What does μονογενής mean?

### Input:

and its grammatical case is accusative. μονογενής is a adj, and its semantic 
domain information is 058007. The grammatical gender of this word is masculine, 
meaning “only begotten.” The lemma form of this word is μονογενής, 
The Louw-Nida semantic domain entry for this word is 58.52. and it is 
classified as a A-ASM and its grammatical number is singular. with a Strong’s 
number of 3439. This word functions within 3 discourse feature(s), including  
Focus+, which is defined as Constituents placed in P2 to give them focal 
prominence. Verb Focus+, which is defined as Verb in final position in clause 
demonstrates verb focus. Main clauses, which is defined as Main clauses are the 
top-level clauses in a sentence. 
They are the clauses that are not embedded in other clauses.


In [ ]:
prompt = generate_prompt('Generate a list of questions based off of the input.', input_ctxt)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to(model.device)

with torch.no_grad():
    outputs = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
    )
response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
print(response)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generage a list of questions based off of the input.

### Input:

and its grammatical case is accusative. μονογενής is a adj, and its semantic 
domain information is 058007. The grammatical gender of this word is masculine, 
meaning “only begotten.” The lemma form of this word is μονογενής, 
The Louw-Nida semantic domain entry for this word is 58.52. and it is 
classified as a A-ASM and its grammatical number is singular. with a Strong’s 
number of 3439. This word functions within 3 discourse feature(s), including  
Focus+, which is defined as Constituents placed in P2 to give them focal 
prominence. Verb Focus+, which is defined as Verb in final position in clause 
demonstrates verb focus. Main clauses, which is defined as Main clauses are the 
top-level clauses in a sentence. 
They are the clauses that are not 